# 1_fina(고객정보데이터) 클러스터링

In [0]:
!pip install kmeans-pytorch


In [0]:
from kmeans_pytorch import kmeans
from pyspark.sql import Row


In [0]:

df_final = spark.read.format("delta").table("database_pjt.1_fina_entire")
df_final.show(3)


In [0]:
df_final.printSchema()
print(df_final.columns)


In [0]:
# 첫 번째 row의 벡터 길이(차원) 확인
df_final.select("scaled_log_features").first()[0].size


In [0]:
#벡터화는 되어있으니 pca는 해야함. 최적의 k값도 찾아야해
from pyspark.ml.feature import PCA

# k: 축소할 차원 수, inputCol: 벡터 컬럼명, outputCol: 결과 컬럼명
pca = PCA(k=5, inputCol="scaled_log_features", outputCol="pca_features")
pca_model = pca.fit(df_final)
df_final = pca_model.transform(df_final)

# 결과 확인
df_final.select("pca_features").show(3, truncate=False)


In [0]:
#드라이버 메모리 부담 최소화를 위해 블럭 나눔
# (1) PySpark에서 10% 샘플링 후, PCA 벡터만 추출
import numpy as np

df_final_sample = df_final.sample(fraction=0.1, seed=42)
pca_sample_np = np.array([row.pca_features.toArray() for row in df_final_sample.select("pca_features").collect()])


In [0]:
# 회원정보 전체 데이터 수 계산 -> 클러스터링 처리 연산 관련 데이터량 추정
df_final.count()

In [0]:
import torch

# 샘플 데이터만 GPU로 변환
pca_sample_tensor = torch.from_numpy(pca_sample_np).float().cuda()


In [0]:
#kmeans(gpu) 샘플데이터로 클러스터링
#이거 하고 최단연결, 최장연결, 평균연결,중심연결 4개 비교도 해보고싶긴한데
from kmeans_pytorch import kmeans

# 샘플 데이터로 KMeans (유클리드)
kmeans_cluster_ids, kmeans_centers = kmeans(
    X=pca_sample_tensor,
    num_clusters=4,
    distance='euclidean',
    device=torch.device('cuda')
)



In [0]:
# 샘플 데이터로 KMeans (코사인)
kmeans2_cluster_ids, kmeans2_centers = kmeans(
    X=pca_sample_tensor,
    num_clusters=5,
    distance='cosine',
    device=torch.device('cuda')
)

In [0]:
# 4. 전체 데이터 예측 (배치 처리 권장, 메모리 부족시 limit 사용)
# 4. 전체 데이터 예측 함수/ 데이터가 어느 클러스터 속하는지 예측할당
import torch
def assign_clusters(X, centers, distance='euclidean'):
    centers = centers.to(X.device)
    if distance == 'euclidean':
        dists = torch.cdist(X, centers)
    elif distance == 'cosine':
        X_norm = X / X.norm(dim=1, keepdim=True)
        centers_norm = centers / centers.norm(dim=1, keepdim=True)
        dists = 1 - torch.mm(X_norm, centers_norm.t())
    else:
        raise ValueError("지원하지 않는 distance")
    return torch.argmin(dists, dim=1)

# 5. 전체 데이터 배치 처리 (70,000개 제한 예시)
df_final_limit = df_final.limit(1000000)
pca_features_np = np.array([row.pca_features.toArray() for row in df_final_limit.select("pca_features").collect()])
pca_features_tensor = torch.from_numpy(pca_features_np).float().cuda()

In [0]:
# 6. 클러스터 할당 즉 행이 어떤 클러스터 속하는지 실제 계산후 라벨링
kmeans_cluster_ids_full = assign_clusters(pca_features_tensor, kmeans_centers, 'euclidean')
kmeans2_cluster_ids_full = assign_clusters(pca_features_tensor, kmeans2_centers, 'cosine')


In [0]:
# 7. 메타 클러스터링/ 유클리드랑 코사인거리등 보고, 통합해서 한번더 군집화
meta_features = torch.stack([kmeans_cluster_ids_full, kmeans2_cluster_ids_full], dim=1).float()
meta_cluster_ids, meta_centers = kmeans(
    X=meta_features,
    num_clusters=3,
    distance='euclidean',
    device=torch.device('cuda'),
    tol=1e-4
)



클러스터는 시험적으로 3개로 하는건가요 ?
5개를 햇었다가 오류났어서 그 오류 수정하는 방식에서 바꿨던거라 
숫자를 더 늘려보긴 할겁니다

알겠습니다!


In [0]:
print(torch.unique(meta_cluster_ids, return_counts=True))


In [0]:
print(torch.isnan(meta_features).any(), torch.isinf(meta_features).any())


In [0]:
# 8. 결과 변환
final_cluster_ids_np = meta_cluster_ids.cpu().numpy().astype(int)
rows = [Row(id=idx, final_cluster=int(cluster)) for idx, cluster in enumerate(final_cluster_ids_np)]
df_final_cluster = spark.createDataFrame(rows)
df_final_cluster.show(5)

In [0]:
import numpy as np
from sklearn.metrics import silhouette_score

# 리미트개수에서 1,0000개만 무작위 샘플링
idx = np.random.choice(len(pca_sample_np), 30000, replace=False)
sample_np = pca_sample_np[idx]
sample_labels = kmeans_cluster_ids.cpu().numpy()[idx]

score = silhouette_score(sample_np, sample_labels, metric='euclidean')
print(f"Silhouette Score (Euclidean, 30000개 샘플): {score:.4f}")


In [0]:
#전체 코드

#드라이버 메모리 부담 최소화를 위해 블럭 나눔
# (1) PySpark에서 10% 샘플링 후, PCA 벡터만 추출
import numpy as np

df_final_sample = df_final.sample(fraction=0.1, seed=42)
pca_sample_np = np.array([row.pca_features.toArray() for row in df_final_sample.select("pca_features").collect()])
import torch

# 샘플 데이터만 GPU로 변환
pca_sample_tensor = torch.from_numpy(pca_sample_np).float().cuda()
#kmeans(gpu) 샘플데이터로 클러스터링
#이거 하고 최단연결, 최장연결, 평균연결,중심연결 4개 비교도 해보고싶긴한데
from kmeans_pytorch import kmeans

# 샘플 데이터로 KMeans (유클리드)
kmeans_cluster_ids, kmeans_centers = kmeans(
    X=pca_sample_tensor,
    num_clusters=7,
    distance='euclidean',
    device=torch.device('cuda')
)

# 샘플 데이터로 KMeans (코사인)
kmeans2_cluster_ids, kmeans2_centers = kmeans(
    X=pca_sample_tensor,
    num_clusters=7,
    distance='cosine',
    device=torch.device('cuda')
)
# 4. 전체 데이터 예측 (배치 처리 권장, 메모리 부족시 limit 사용)
# 4. 전체 데이터 예측 함수/ 데이터가 어느 클러스터 속하는지 예측할당
import torch
def assign_clusters(X, centers, distance='euclidean'):
    centers = centers.to(X.device)
    if distance == 'euclidean':
        dists = torch.cdist(X, centers)
    elif distance == 'cosine':
        X_norm = X / X.norm(dim=1, keepdim=True)
        centers_norm = centers / centers.norm(dim=1, keepdim=True)
        dists = 1 - torch.mm(X_norm, centers_norm.t())
    else:
        raise ValueError("지원하지 않는 distance")
    return torch.argmin(dists, dim=1)

# 5. 전체 데이터 배치 처리 (10,000개 제한 예시)
#df_final_limit = df_final.limit(10000)
#pca_features_np = np.array([row.pca_features.toArray() for row in df_final_limit.select("pca_features").collect()])
#pca_features_tensor = torch.from_numpy(pca_features_np).float().cuda()
# 6. 클러스터 할당 즉 행이 어떤 클러스터 속하는지 실제 계산후 라벨링
kmeans_cluster_ids_full = assign_clusters(pca_features_tensor, kmeans_centers, 'euclidean')
kmeans2_cluster_ids_full = assign_clusters(pca_features_tensor, kmeans2_centers, 'cosine')

# 7. 메타 클러스터링/ 유클리드랑 코사인거리등 보고, 통합해서 한번더 군집화
meta_features = torch.stack([kmeans_cluster_ids_full, kmeans2_cluster_ids_full], dim=1).float()
meta_cluster_ids, meta_centers = kmeans(
    X=meta_features,
    num_clusters=3,
    distance='euclidean',
    device=torch.device('cuda'),
    tol=1e-4
)



In [0]:
from kmeans_pytorch import kmeans
from pyspark.sql import Row
import numpy as np
import torch

# 1. 10% 샘플링 후, PCA 벡터만 추출
df_final_sample = df_final.sample(fraction=0.1, seed=42)
pca_sample_np = np.array([row.pca_features.toArray() for row in df_final_sample.select("pca_features").collect()])
pca_sample_tensor = torch.from_numpy(pca_sample_np).float().cuda()

# 2. 샘플 데이터로 KMeans (유클리드 거리만)
kmeans_cluster_ids, kmeans_centers = kmeans(
    X=pca_sample_tensor,
    num_clusters=5,  # 원하는 군집 개수
    distance='euclidean',
    device=torch.device('cuda')
)

# 3. 전체 데이터(10,000개 제한) 클러스터 할당
df_final_limit = df_final.limit(1000000)
pca_features_np_limit = np.array([row.pca_features.toArray() for row in df_final_limit.select("pca_features").collect()])
pca_features_tensor_limit = torch.from_numpy(pca_features_np_limit).float().cuda()

# 유클리드 거리로 각 행의 클러스터 할당
kmeans_cluster_ids_full = torch.argmin(torch.cdist(pca_features_tensor_limit, kmeans_centers.to(pca_features_tensor_limit.device)), dim=1)

# 4. 결과 변환
final_cluster_ids_np = kmeans_cluster_ids_full.cpu().numpy().astype(int)
rows = [Row(id=idx, final_cluster=int(cluster)) for idx, cluster in enumerate(final_cluster_ids_np)]
df_final_cluster = spark.createDataFrame(rows)
df_final_cluster.show(5)


In [0]:
import numpy as np
from sklearn.metrics import silhouette_score

# 리미트개수에서 1,0000개만 무작위 샘플링
idx = np.random.choice(len(pca_sample_np), 30000, replace=False)
sample_np = pca_sample_np[idx]
sample_labels = kmeans_cluster_ids.cpu().numpy()[idx]

score = silhouette_score(sample_np, sample_labels, metric='euclidean')
print(f"Silhouette Score (Euclidean, 30000개 샘플): {score:.4f}")


## 분석 방향성

- 군집의 분포 및 특성 분석
 
군집별 데이터 수(분포)
각 군집에 데이터가 얼마나 분포되어 있는지 확인. 한쪽 군집에 쏠림이 심하면 해석에 주의해야 합니다.
 
군집 중심(centroid) 값 해석
각 군집의 중심값(평균 벡터)을 확인하여, 군집별 대표적 특성(변수별 평균, 패턴 등)을 파악합니다.
 
군집별 주요 변수 요약
군집별로 주요 피처(변수)의 평균, 분산, 범위 등을 요약해 군집의 특성을 설명합니다.
 
- 군집 간/내 거리 및 분리도
 
군집 내 거리(Compactness)
각 군집 내부의 데이터들이 얼마나 밀집해 있는지(=군집 내 거리의 평균/분산).
 
군집 간 거리(Separation)
서로 다른 군집 중심점(centroid) 간의 거리. 군집 간 거리가 크면 클수록 군집이 잘 분리된 것.

## 시각화

 
- 차원 축소 후 시각화(PCA) 2~3차원으로 차원 축소 후, 군집별로 색을 달리해 데이터 분포와 군집 경계를 시각적으로 확인합니다.
 

In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
pca_features_np_limit_scaled = scaler.fit_transform(pca_features_np_limit)


In [0]:
import matplotlib.pyplot as plt
from pyspark.sql.functions import col
from sklearn.decomposition import PCA
import pyspark.sql.functions as F


# 클러스터링 결과를 원본 df_final_limit와 join
df_with_cluster = df_final_limit.withColumn("row_id", F.monotonically_increasing_id())
df_final_cluster_with_index = df_final_cluster.withColumnRenamed("id", "row_id")
df_joined = df_with_cluster.join(df_final_cluster_with_index, on="row_id")

# PCA 2D 적용을 위한 numpy array 추출
pca_features_np_vis = np.array([row.pca_features.toArray() for row in df_joined.select("pca_features").collect()])
cluster_ids_vis = np.array([row.final_cluster for row in df_joined.select("final_cluster").collect()])

# PCA 차원 축소 (2D)
pca_2d = PCA(n_components=2)
pca_2d_result = pca_2d.fit_transform(pca_features_np_vis)

# 시각화
plt.figure(figsize=(10, 7))
for cluster_id in np.unique(cluster_ids_vis):
    idxs = cluster_ids_vis == cluster_id
    plt.scatter(pca_2d_result[idxs, 0], pca_2d_result[idxs, 1], label=f'Cluster {cluster_id}', alpha=0.6)

plt.title('PCA 2D Visualization of Clusters')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.legend()
plt.grid(True)
plt.show()


In [0]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import numpy as np
from mpl_toolkits.mplot3d import Axes3D  # 꼭 필요합니다

# pca_features_np_limit: (1000000, n_features)
# final_cluster_ids_np: (1000000,)

# 1. 3차원 PCA로 축소
pca_3d = PCA(n_components=3)
pca_3d_result = pca_3d.fit_transform(pca_features_np_limit)

# 2. 3만 개 샘플링
sample_idx = np.random.choice(len(pca_3d_result), 30000, replace=False)
pca_3d_sample = pca_3d_result[sample_idx]
cluster_sample = final_cluster_ids_np[sample_idx]

# 3. 시각화
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

for cluster_id in np.unique(cluster_sample):
    idx = (cluster_sample == cluster_id)
    ax.scatter(
        pca_3d_sample[idx, 0],
        pca_3d_sample[idx, 1],
        pca_3d_sample[idx, 2],
        s=10,
        color=colors[cluster_id % len(colors)],
        label=f'Cluster {cluster_id}',
        alpha=0.6
    )

ax.set_title('3D PCA Scatter Plot by Cluster')
ax.set_xlabel('PCA Component 1')
ax.set_ylabel('PCA Component 2')
ax.set_zlabel('PCA Component 3')
ax.legend()
plt.show()


3D PCA 군집 시각화 해석

이 그래프는 전체 데이터(100만 건) 중 3만 건을 무작위로 샘플링한 뒤,  
PCA로 3차원 축소하여 각 군집별로 색을 다르게 표시한 3D 산점도입니다.

---

주요 해석

- **군집 0 (파란색)**
  - 3D 공간에서 가장 넓고 오른쪽 영역에 분포
  - 데이터가 많이 몰려 있고, 퍼짐이 큼
  - 전체적으로 주요 분산 방향을 따라 넓게 펼쳐져 있음

- **군집 1 (주황색)**
  - 위쪽 평면에 띠 형태로 뚜렷하게 분리되어 있음
  - 다른 군집과 거의 중첩 없이 독립적으로 존재
  - 응집도가 높고, 경계가 명확함

- **군집 2 (초록색), 군집 3 (빨간색), 군집 4 (보라색)**
  - 왼쪽 및 중앙 하단 영역에 위치
  - 군집 2(초록)와 군집 3(빨강)은 나란히 붙어 있고, 군집 4(보라)는 이들과 일부 중첩됨
  - 군집 3은 특히 한정된 공간에 조밀하게 모여 있음

---

전체적 분포 및 군집 품질

- **군집 간 분리도(Separation)**
  - 3D 공간에서 각 군집이 명확히 분리되어 있음
  - 군집 0과 군집 1은 서로 뚜렷하게 떨어져 있음
  - 군집 2, 3, 4는 경계가 일부 맞닿거나 중첩되어 있지만, 대체로 구분 가능

- **군집 내 응집도(Compactness)**
  - 군집 3, 2는 내부 데이터가 조밀하게 모여 있음
  - 군집 0, 1은 상대적으로 넓게 퍼져 있음

- **분석 결론**
  - 고차원 데이터의 군집 구조가 3D에서도 잘 보존됨
  - 군집별 특성이 뚜렷하게 구분되는 구조
  - 군집 0(파랑)이 가장 큰 집단이며, 군집 1(주황)은 명확한 띠 형태, 나머지 군집은 각각 특정 영역에 분포



- 3D PCA 시각화는 2D보다 군집 간 분리와 내부 구조를 더 잘 보여줌
- 군집별 특성(프로파일링) 분석을 통해 각 군집이 실제로 어떤 속성을 갖는지 추가 해석 필요
- PCA는 데이터의 주요 분산 방향만 보존하므로, 실제 고차원 구조와 일부 차이가 있을 수 있음

---

**요약:**  
각 군집이 3D 공간에서 명확하게 분리되어 있고, 군집별 분포와 경계가 뚜렷하게 나타남.  
군집 0이 가장 크고, 군집 1은 띠 형태로 독립적, 군집 2·3·4는 일부 중첩되어 있으나 대체로 구분 가능.  
전체적으로 군집화 품질이 우수하게 시각화된 결과임.


In [0]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import numpy as np
 
# pca_features_np_limit: (1000000, n_features) - 전체 데이터의 PCA 벡터 (numpy 배열)
# final_cluster_ids_np: (1000000,) - 각 데이터의 군집 ID (numpy 배열)
 
# 1. 2차원 PCA로 추가 축소
pca_2d = PCA(n_components=2)
pca_2d_result = pca_2d.fit_transform(pca_features_np_limit)
 
# 2. 3만개 샘플링 (시각화 성능 및 가독성 위해)
sample_idx = np.random.choice(len(pca_2d_result), 30000, replace=False)
pca_2d_sample = pca_2d_result[sample_idx]
cluster_sample = final_cluster_ids_np[sample_idx]

# PCA 1, 2에 가장 기여한 상위 3개 feature 추출
top_features_pc1 = np.argsort(np.abs(pca_2d.components_[0]))[::-1][:3]
top_features_pc2 = np.argsort(np.abs(pca_2d.components_[1]))[::-1][:3]

print("PCA Component 1 주요 feature:", [feature_names[i] for i in top_features_pc1])
print("PCA Component 2 주요 feature:", [feature_names[i] for i in top_features_pc2])
 
xlabel = "PCA Component 1\n(Main: " + ", ".join([feature_names[i] for i in top_features_pc1]) + ")"
ylabel = "PCA Component 2\n(Main: " + ", ".join([feature_names[i] for i in top_features_pc2]) + ")"

# 3. 군집별 색상 지정
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
 
plt.figure(figsize=(10, 8))
for cluster_id in np.unique(cluster_sample):
    idx = (cluster_sample == cluster_id)
    plt.scatter(
        pca_2d_sample[idx, 0],
        pca_2d_sample[idx, 1],
        s=10,
        color=colors[cluster_id],
        label=f'Cluster {cluster_id}',
        alpha=0.6
    )
 
plt.title('2D PCA Scatter Plot by Cluster')
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.legend()
plt.grid(True)
plt.show()
 
 

In [0]:
pip install umap-learn

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql.functions import col
import umap.umap_ as umap  # pip install umap-learn

# 1. PySpark DataFrame에서 NumPy 배열로 변환
pca_features_np_vis = np.array([row.pca_features.toArray() for row in df_joined.select("pca_features").collect()])
cluster_ids_vis = np.array([row.final_cluster for row in df_joined.select("final_cluster").collect()])

# 2. UMAP 적용 (2차원으로 차원 축소)
umap_2d = umap.UMAP(n_components=2)
umap_2d_result = umap_2d.fit_transform(pca_features_np_vis)

# 3. 시각화
plt.figure(figsize=(10, 7))
for cluster_id in np.unique(cluster_ids_vis):
    idxs = cluster_ids_vis == cluster_id
    plt.scatter(umap_2d_result[idxs, 0], umap_2d_result[idxs, 1], label=f'Cluster {cluster_id}', alpha=0.6)

plt.title('UMAP 2D Visualization of Clusters')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.legend()
plt.grid(True)
plt.show()


- 군집별 대표 샘플 추출
각 군집 중심에 가장 가까운 데이터(대표 샘플)를 뽑아 실제 특성을 확인합니다.

In [0]:
import pandas as pd

# 클러스터 중심 좌표
centers_np = kmeans_centers.cpu().numpy()

# 모든 데이터 벡터
features_np_all = pca_features_tensor_limit.cpu().numpy()

# 각 샘플과 중심 거리 계산
distances = np.linalg.norm(features_np_all[:, None, :] - centers_np[None, :, :], axis=2)

# 가장 가까운 점 인덱스 추출 (각 클러스터별)
representative_idxs = np.argmin(distances, axis=0)  # shape: (num_clusters,)

# Spark DataFrame에서 대표 샘플 추출
df_with_row_idx = df_final_limit.withColumn("row_id", F.monotonically_increasing_id())
representative_rows = df_with_row_idx.filter(F.col("row_id").isin([int(i) for i in representative_idxs]))

display(representative_rows.show(truncate=False))


## 결과분석

### 군집화 품질 평가

데이터 분포 확인 및 군집 내 / 외 거리, 실루엣 점수 

In [0]:
import numpy as np

# final_cluster_ids_np는 이미 100만개 데이터에 대해 군집 ID가 할당된 numpy 배열
# 군집 개수는 5로 가정
unique, counts = np.unique(final_cluster_ids_np, return_counts=True)
for cluster_id, count in zip(unique, counts):
    print(f"군집 {cluster_id}: {count}개")


 1. 군집별 데이터 분포
- **가장 큰 군집**: 군집 0 (약 32만 개, 전체의 약 32%)
- **가장 작은 군집**: 군집 1 (약 8만 개, 전체의 약 8%)
- **나머지 군집**: 군집 2, 3, 4는 각각 16~22% 비중을 차지
---
 2. 해석
- **불균형 분포**  
  군집별 데이터 개수가 균등하지 않고, 군집 0에 데이터가 많이 몰려 있음.  
  군집 1은 상대적으로 매우 적은 데이터가 할당됨.
- **의미**  
  - 군집 0은 데이터에서 가장 흔한(대표적인) 특성을 가진 집단일 가능성이 높음
  - 군집 1은 특이하거나 극단적인 특성을 가진 소수 집단일 수 있음
  - 군집 2, 3, 4는 중간 규모의 집단으로, 각각 다른 특성의 하위 그룹일 가능성

In [0]:
# kmeans_centers: (5, n_features) 텐서
# pca_features_np_limit: (1000000, n_features) numpy 배열
# final_cluster_ids_np: (1000000,) numpy 배열

# 각 데이터별로 본인 군집의 중심점과 거리 계산
distances = np.linalg.norm(
    pca_features_np_limit - kmeans_centers.cpu().numpy()[final_cluster_ids_np], axis=1
)
inertia = np.sum(distances ** 2)
print(f"Inertia(SSE): {inertia:.2f}")


In [0]:
from scipy.spatial.distance import cdist

# 군집 간 중심점 거리 행렬
center_distances = cdist(kmeans_centers.cpu().numpy(), kmeans_centers.cpu().numpy())
print("군집 간 중심점 거리 행렬:")
print(center_distances)


 군집 간 중심점 거리 행렬 해석

|       | 군집 0 | 군집 1 | 군집 2 | 군집 3 | 군집 4 |
|-------|--------|--------|--------|--------|--------|
| 군집 0| 0.00   | 4.94   | 3.37   | 3.01   | 3.29   |
| 군집 1| 4.94   | 0.00   | 3.88   | 4.08   | 3.14   |
| 군집 2| 3.37   | 3.88   | 0.00   | 1.95   | 2.13   |
| 군집 3| 3.01   | 4.08   | 1.95   | 0.00   | 1.60   |
| 군집 4| 3.29   | 3.14   | 2.13   | 1.60   | 0.00   |

---

#### 1. 해석

- **군집 중심점 간 거리**는 각 군집의 중심(centroid) 벡터들 사이의 유클리드 거리입니다.
- **가장 멀리 떨어진 군집**: 군집 0과 군집 1 (4.94)
- **가장 가까운 군집**: 군집 3과 군집 4 (1.60), 군집 2와 군집 3 (1.95)
- **의미**:  
  - 거리가 클수록 해당 군집들이 데이터 특성상 명확하게 분리되어 있다는 뜻.
  - 거리가 작을수록 두 군집이 비슷한 특성을 가질 가능성이 높음.
---

#### 2. 활용 방향

- 군집 간 거리가 충분히 크면 군집화가 잘 분리된 것.
- 거리가 작은 군집 쌍(예: 군집 3-4, 2-3)은 특성이 유사할 수 있으니 군집별 특성 분석 시 주의.
- 군집별 대표 특성, 변수 평균 등을 추가로 비교하면 더 명확한 해석 가능.

---


In [0]:
for cluster_id in range(5):
    idxs = np.where(final_cluster_ids_np == cluster_id)[0]
    cluster_points = pca_features_np_limit[idxs]
    center = kmeans_centers.cpu().numpy()[cluster_id]
    mean_dist = np.mean(np.linalg.norm(cluster_points - center, axis=1))
    print(f"군집 {cluster_id} 내 평균 거리: {mean_dist:.4f}")


In [0]:
from sklearn.metrics import silhouette_score

# pca_features_np_limit: (1000000, n_features)
# final_cluster_ids_np: (1000000,)
sample_idx = np.random.choice(len(pca_features_np_limit), 30000, replace=False)
sample_np = pca_features_np_limit[sample_idx]
sample_labels = final_cluster_ids_np[sample_idx]
score = silhouette_score(sample_np, sample_labels, metric='euclidean')
print(f"Silhouette Score (Euclidean, 3만개 샘플): {score:.4f}")


In [0]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import numpy as np

# pca_features_np_limit: (1000000, n_features) - 전체 데이터의 PCA 벡터 (numpy 배열)
# final_cluster_ids_np: (1000000,) - 각 데이터의 군집 ID (numpy 배열)

# 1. 2차원 PCA로 추가 축소
pca_2d = PCA(n_components=2)
pca_2d_result = pca_2d.fit_transform(pca_features_np_limit)

# 2. 3만개 샘플링 (시각화 성능 및 가독성 위해)
sample_idx = np.random.choice(len(pca_2d_result), 30000, replace=False)
pca_2d_sample = pca_2d_result[sample_idx]
cluster_sample = final_cluster_ids_np[sample_idx]

# 3. 군집별 색상 지정
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

plt.figure(figsize=(10, 8))
for cluster_id in np.unique(cluster_sample):
    idx = (cluster_sample == cluster_id)
    plt.scatter(
        pca_2d_sample[idx, 0], 
        pca_2d_sample[idx, 1], 
        s=10, 
        color=colors[cluster_id], 
        label=f'Cluster {cluster_id}', 
        alpha=0.6
    )

plt.title('2D PCA Scatter Plot by Cluster')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.legend()
plt.grid(True)
plt.show()


2D PCA 군집 시각화 해석
이 그래프는 전체 데이터(100만 건)에서 3만 건을 무작위로 샘플링한 뒤,  
PCA로 2차원 축소하여 각 군집(클러스터)별로 색을 다르게 표시한 산점도입니다.

주요 해석
- **군집 0 (파란색)**  
  - 오른쪽에 가장 넓게 분포
  - 데이터가 가장 많이 몰려 있고, 분산(퍼짐)도 큼
  - PCA 주성분 기준으로, 이 군집이 가장 큰 집단임을 시각적으로 확인 가능

- **군집 1 (주황색)**  
  - 왼쪽 상단에 뚜렷하게 분리되어 있음
  - 경계가 명확하고, 비교적 응집도가 높음
  - 다른 군집과 중첩이 거의 없음

- **군집 2 (초록색)**  
  - 왼쪽 하단에 위치
  - 데이터가 선형적으로 퍼진 형태가 보임
  - 군집 1과 일부 경계가 맞닿아 있지만, 군집 0과는 명확히 분리

- **군집 3 (빨간색)**  
  - 중앙 하단에 위치
  - 비교적 작은 영역에 조밀하게 모여 있음
  - 군집 2, 4와 경계가 맞닿아 있음

- **군집 4 (보라색)**  
  - 중앙~좌상단에 분포
  - 군집 1, 2, 3과 경계가 일부 중첩
  - 분산이 크고, 다양한 방향으로 퍼져 있음

---
전체적 분포 및 군집 품질

- **군집 간 분리도(Separation)**  
  - 각 군집은 PCA 2D 공간에서 명확히 분리되어 있음
  - 특히 군집 0(파랑)과 군집 1(주황)은 서로 뚜렷하게 떨어져 있음

- **군집 내 응집도(Compactness)**  
  - 군집 3, 2는 내부 데이터가 조밀하게 모여 있음
  - 군집 0, 4는 상대적으로 퍼져 있음

- **경계 중첩**  
  - 군집 4는 여러 군집과 경계가 일부 겹침. PCA 축소 과정에서 정보 손실로 인한 현상일 수 있음

- **분석 결론**  
  - 고차원 데이터의 군집 구조가 2D에서도 잘 보존됨
  - 군집별로 특성이 뚜렷하게 구분되는 구조
  - 군집 0(파랑)이 가장 큰 집단이고, 군집 1(주황), 2(초록), 3(빨강), 4(보라)는 각각 특정 영역에 분포

---
- PCA는 데이터의 주요 분산 방향만 2D로 투영하므로, 실제 고차원 구조와 일부 차이가 있을 수 있음
- 군집별 특성(프로파일링) 분석을 통해 각 군집이 실제로 어떤 속성을 갖는지 추가 해석 필요
**요약:**  
각 군집이 2D 공간에서 명확하게 분리되어 있고, 군집별 분포와 경계가 뚜렷하게 나타남.  
군집 0이 가장 크고, 군집 1과 2는 경계가 명확, 군집 4는 여러 군집과 일부 중첩되어 있음.  
전체적으로 군집화 품질이 우수하게 시각화된 결과임.


In [0]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=['log_입회경과개월수_신용', 'log__1순위카드이용금액', 'log__1순위카드이용건수', 'log__2순위카드이용금액', 'log__2순위카드이용건수'],
    outputCol='scaled_log_features'
)


In [0]:
# PCA 모델의 주성분 가중치 행렬 추출 (k=5, n_features=5)
pc_weights = pca_model.pc.toArray()  # shape: (5, 5)

# 실제 inputCols 변수명과 매칭
input_cols = ['log_입회경과개월수_신용', 'log__1순위카드이용금액', 'log__1순위카드이용건수', 'log__2순위카드이용금액', 'log__2순위카드이용건수']

for i, component in enumerate(pc_weights):
    print(f"\nPCA Component {i+1}:")
    for var, weight in zip(input_cols, component):
        print(f"{var}: {weight:.4f}")


In [0]:
import numpy as np
from scipy.spatial.distance import cdist

# 군집 중심점 (kmeans_centers): (num_clusters, n_features)
# 전체 데이터의 PCA 벡터 (pca_features_np_limit): (n_samples, n_features)
# 각 데이터의 군집 ID (final_cluster_ids_np): (n_samples,)

num_clusters = kmeans_centers.shape[0]

# 1. 군집 내 거리(Compactness): 각 군집별로 중심점과의 평균 거리
intra_distances = []
for cluster_id in range(num_clusters):
    idxs = np.where(final_cluster_ids_np == cluster_id)[0]
    cluster_points = pca_features_np_limit[idxs]
    center = kmeans_centers.cpu().numpy()[cluster_id]
    # 각 점과 중심점 사이의 거리
    dists = np.linalg.norm(cluster_points - center, axis=1)
    mean_dist = np.mean(dists)
    std_dist = np.std(dists)
    intra_distances.append((cluster_id, mean_dist, std_dist))
    print(f"군집 {cluster_id} 내 평균 거리: {mean_dist:.4f}, 표준편차: {std_dist:.4f}")

# 2. 군집 간 거리(Separation): 중심점들 사이의 거리 행렬
center_distances = cdist(kmeans_centers.cpu().numpy(), kmeans_centers.cpu().numpy())
print("\n군집 간 중심점 거리 행렬:")
print(center_distances)

# 3. 군집 간 최소/최대/평균 거리 요약
upper_tri = center_distances[np.triu_indices(num_clusters, k=1)]
print(f"\n군집 간 최소 거리: {upper_tri.min():.4f}")
print(f"군집 간 최대 거리: {upper_tri.max():.4f}")
print(f"군집 간 평균 거리: {upper_tri.mean():.4f}")


###

In [0]:
print(df_with_cluster.columns)


In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql.types import DoubleType, IntegerType, LongType, FloatType

# 1. row_id 컬럼이 없으면 생성
if 'row_id' not in df_final_limit.columns:
    df_final_limit = df_final_limit.withColumn('row_id', F.monotonically_increasing_id())

# 2. 군집 ID DataFrame 생성
rows = [Row(row_id=int(idx), cluster=int(cluster)) for idx, cluster in enumerate(final_cluster_ids_np)]
df_cluster = spark.createDataFrame(rows)

# 3. row_id로 join
df_with_cluster = df_final_limit.join(df_cluster, on='row_id')

# 4. 수치형 컬럼 자동 추출 (row_id, cluster 제외)
numeric_cols = [
    f.name for f in df_with_cluster.schema.fields
    if isinstance(f.dataType, (DoubleType, IntegerType, LongType, FloatType))
    and f.name not in ['cluster', 'row_id']
]

# 5. 집계 함수 생성
agg_exprs = []
for c in numeric_cols:
    agg_exprs.append(F.mean(c).alias(f'mean_{c}'))
    agg_exprs.append(F.stddev(c).alias(f'std_{c}'))
    agg_exprs.append(F.min(c).alias(f'min_{c}'))
    agg_exprs.append(F.max(c).alias(f'max_{c}'))

# 6. 군집별 통계 집계
result = df_with_cluster.groupBy('cluster').agg(*agg_exprs)
result.show(truncate=False)


In [0]:
!pip install tabulate

In [0]:
!pip install fpdf


In [0]:
!pip install reportlab


In [0]:
from tabulate import tabulate


In [0]:
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas

pdf_result = result.toPandas()

c = canvas.Canvas("cluster_profile.pdf", pagesize=landscape(letter))
c.setFont("Helvetica", 8)

row_height = 15
x_offset = 40
y_offset = 550
col_width = 80

# 컬럼명 출력
for i, col in enumerate(pdf_result.columns):
    c.drawString(x_offset + i * col_width, y_offset, str(col))

# 데이터 출력
for row_idx, row in enumerate(pdf_result.values):
    for col_idx, value in enumerate(row):
        c.drawString(x_offset + col_idx * col_width, y_offset - row_height * (row_idx + 1), str(value))

c.save()


In [0]:
pdf_file_path = "cluster_profile.pdf"
pdf_file_path


In [0]:
import os

pdf_file_path = "cluster_profile.pdf"
file_exists = os.path.isfile(pdf_file_path)
print(file_exists)  # True면 파일 있음, False면 없음


# 1 클러스터링결과 상세분석


## 📊 군집별 특성 분석 결과 (Clusters Characteristics Analysis)

제공된 Spark DataFrame 집계 결과는 K-Means 군집화 이후 각 군집(Cluster)이 어떤 통계적 특성을 가지는지 보여주는 매우 중요한 자료입니다.  
이를 통해 각 군집의 '페르소나'를 파악하고 비즈니스적 의미를 도출할 수 있습니다.

아래는 각 군집의 주요 특징을 요약하고 해석한 내용입니다.  
이 결과를 바탕으로 군집의 의미를 이해하고, 비즈니스 전략에 활용할 수 있습니다.

---

## 군집 0: **오래된 VIP & 활동성 높은 일반 고객**
- **규모:** 가장 큰 규모 (319,498명)
- **주요 특징:**
  - **오랜 고객:** 입회경과개월수_신용 (평균 84개월), 최종카드발급경과월 (평균 17.5개월) 모두 가장 오래된 고객임을 시사
  - **활발한 이용:** 소지카드수_유효_신용 (평균 1.74개), 이용카드수_신용 (평균 2.26개)이 가장 많음
  - **높은 지출:** 총이용금액 (평균 339만) 및 관련 log_ 변수들이 다른 군집에 비해 가장 높음
  - **낮은 이탈 이력:** 탈회횟수_누적 (평균 0.47회) 등 탈회 이력이 낮은 편
  - **VIP 비중:** VIP등급코드_idx (평균 0.56)가 다른 군집에 비해 높음
- **해석:**  
  오랜 기간 서비스를 이용하며, 활발하게 카드를 소지/이용하고 높은 지출을 하는 충성 고객이 다수 포함된 군집입니다. VIP 고객 비중도 높아 이탈 방지 및 프리미엄 서비스 제공이 중요합니다.

---

## 군집 1: **저활동/초기 이탈 가능성 고객**
- **규모:** 중간 규모 (80,700명)
- **주요 특징:**
  - **오래된 발급:** 최종카드발급경과월 (평균 25.4개월)은 군집 0, 4보다 길어, 최근에 카드를 발급받았을 가능성이 낮음
  - **극도로 낮은 이용:** 이용카드수_신용 (평균 0.014개), 총이용금액 (평균 261원) 모두 매우 낮음
  - **일부 탈회 이력:** 탈회횟수_누적 (평균 0.52회) 등 탈회 이력이 있음
- **해석:**  
  카드를 소지하고 있지만 거의 사용하지 않거나, 과거에 탈회 이력이 있고 현재도 비활성 상태인 고객이 많음. 잠재적 이탈 고객이거나 유령 고객일 가능성이 높으므로, 재활성화 캠페인 또는 효율적 관리를 위한 전략이 필요합니다.

---

## 군집 2: **신규 고객 또는 빠르게 이탈한 고객**
- **규모:** 중간 규모 (217,678명)
- **주요 특징:**
  - **최근 가입:** 입회경과개월수_신용 (평균 7.78개월), 최종카드발급경과월 (평균 11.3개월) 모두 가장 짧음
  - **높은 카드 신청:** 카드신청건수 (평균 0.28건)가 가장 높음
  - **높은 탈회 이력:** 탈회횟수_누적 (평균 0.70회), 탈회횟수_발급6개월이내 (평균 0.056회) 등 이탈 이력이 가장 높음
- **해석:**  
  최근에 가입했거나 카드 발급 후 빠르게 이탈한 이력이 있는 고객들이 많음. 신규 고객이면서도 이탈 위험이 높은 그룹으로, 온보딩 프로세스 강화 및 초기 이탈 방지 프로그램이 시급합니다.

---

## 군집 3: **적극적인 신규 또는 중급 이용 고객**
- **규모:** 중간 규모 (223,079명)
- **주요 특징:**
  - **오랜 고객 & 중간 이용:** 입회경과개월수_신용 (평균 108.45개월)이 가장 길고, 총이용금액 (평균 198만) 및 관련 log_ 변수들이 군집 0 다음으로 높음
  - **가장 낮은 탈회 이력:** 탈회횟수_누적 (평균 0.39회)이 가장 낮음
- **해석:**  
  오랜 기간 고객이었지만, 군집 0만큼의 이용 빈도나 금액은 아니지만 꾸준히 카드를 이용하는 고객이 많음. 탈회 이력이 가장 적어 비교적 안정적인 고객군으로, VIP 군집(군집 0)으로 성장시킬 수 있는 잠재력이 있습니다.

---

## 군집 4: **중저가 이용/평범한 고객**
- **규모:** 가장 작은 규모 (159,045명)
- **주요 특징:**
  - **오랜 고객 & 낮은 이용:** 입회경과개월수_신용 (평균 100.67개월)이 군집 3과 유사하게 오래된 고객이나, 총이용금액 (평균 52만) 및 관련 log_ 변수들이 군집 1 다음으로 가장 낮음
  - **낮은 탈회 이력:** 탈회횟수_누적 (평균 0.42회)가 군집 3 다음으로 낮음
- **해석:**  
  오랜 기간 고객이었지만, 카드 이용량이나 금액이 낮은 '평범한' 또는 '잠자는' 고객일 수 있음. 이탈 가능성은 낮지만, 추가적인 가치 창출을 위해 맞춤형 프로모션이나 혜택을 통한 이용 유도가 필요합니다.

---

## 비즈니스적 시사점 및 다음 단계

이러한 군집별 특성 분석은 단순히 숫자를 보는 것을 넘어, 각 군집이 어떤 고객층을 대표하는지 이해하는 데 큰 도움이 됩니다.

- **군집 0:** 최우수/충성 고객 → 이탈 방지 및 프리미엄 서비스 유지 전략
- **군집 1:** 비활성/이탈 가능성 높은 고객 → 재활성화 캠페인 또는 효율적 고객 관리
- **군집 2:** 신규/초기 이탈 위험 고객 → 초기 고객 경험 개선 및 이탈 방지 프로그램 강화
- **군집 3:** 안정적인 중급 이용 고객 → VIP로의 성장 유도를 위한 상향 판매/교차 판매 기회 모색
- **군집 4:** 저활동 장기 고객 → 추가적인 이용 유도를 위한 맞춤형 혜택 제공

이 분석 결과는 이전에 PCA 시각화에서 보았던 군집 4의 넓은 분포(그리고 데이터 개수는 가장 크지 않음)가, 실제로 이 군집이 '다양한' 특성을 가진 '평범한' 고객을 포함하고 있기 때문일 가능성을 시사합니다.  
즉, 시각적으로 넓게 퍼진 것이 단순히 차원 축소의 왜곡일 수도 있지만, 군집 4 내의 데이터들이 특정 특징으로 강하게 뭉치기보다 전반적으로 넓게 분포하는 경향이 있음을 통계적으로 보여주는 것입니다.

---

**다음 단계:**  
이러한 통계적 특성 분석과 시각화 결과를 종합하여 군집화 모델의 적합성을 최종적으로 판단하고,  
필요하다면 군집 개수(K)를 재조정하거나 다른 군집화 알고리즘을 시도해 볼 수 있습니다.


## ✅ 군집별 소비 패턴 해석 및 펀드 추천 전략

## 1. PCA 축 해석 및 소비 패턴 요약

- **PCA1:** 소비 횟수 중심 지표 (값이 클수록 소비 횟수 낮음)
- **PCA2:** 소비 금액 중심 지표 (값이 클수록 소비 금액 높음)

---

## 2. 클러스터별 소비 해석 및 펀드 추천

### 🔹 클러스터 0
- **PCA1:** 0 ~ 3.5 (소비 횟수 적음)
- **PCA2:** -2 ~ 2.5 (금액 중간~낮음)
- **소비 해석:**  
  소비 빈도와 금액이 모두 낮은 편. 카드 사용이 적은 비활동 고객, 또는 특정 시기만 사용하는 소극적 소비자.
- **추천 펀드:**  
  안정성 중심, 로우볼, TDF, 국공채, MMF  
  → 클러스터 2/7번(실버·연금형, 안정지향형)과 유사

---

### 🔹 클러스터 1
- **PCA1:** -3.5 ~ -1.7 (소비 횟수 많음)
- **PCA2:** -0.5 ~ 3.5 (금액도 많음)
- **소비 해석:**  
  활발한 소비자. 횟수와 금액 모두 많음. 카드 중심의 생활형 소비자, 고소득 또는 재무 관심이 큰 고객.
- **추천 펀드:**  
  테마형, 글로벌 투자형, 성장주, 로보어드바이저, 퀀트, 커버드콜  
  → 클러스터 1/5번(트렌드 소비형, 투자 관심형)과 유사

---

### 🔹 클러스터 2
- **PCA1:** -2.5 ~ 1 (보통 횟수)
- **PCA2:** -3 ~ -0.5 (소비 금액 낮음)
- **소비 해석:**  
  사용 횟수는 보통이나 지출 금액이 매우 적음. 가성비·실속형, 사회 초년생 가능성.
- **추천 펀드:**  
  ETF, 중소형주, 단기 투자형  
  → 클러스터 4번(젊은 실속형)과 유사

---

### 🔹 클러스터 3
- **PCA1:** -0.8 ~ 0.5 (횟수 중간)
- **PCA2:** -1.5 ~ 1 (금액도 중간)
- **소비 해석:**  
  일반적인 소비 성향, 중산층 패턴. 일정한 생활 리듬 내 소비, 가족 소비 가능성.
- **추천 펀드:**  
  혼합형펀드, ESG, 자산배분형  
  → 클러스터 6번(가족 중심형)과 유사

---

### 🔹 클러스터 4
- **PCA1:** -2 ~ -0.5 (소비 횟수 많음)
- **PCA2:** -0.5 ~ 2.5 (금액도 보통 이상)
- **소비 해석:**  
  소비 빈도 많고 금액도 적당. 활발한 생활 소비자, 젊은 소비층, 디지털 서비스·배달앱 등 사용률 높음.
- **추천 펀드:**  
  ETF, 중소형주, 팩터, 벤처, 공모주  
  → 클러스터 4번(젊은 실속형) 또는 1번(트렌드 소비형)과 유사

---

## 3. Spark DataFrame 기반 군집별 통계적 특성 요약

### 📊 군집별 특성 분석 결과

- **군집 0: 오래된 VIP & 활동성 높은 일반 고객**  
  - 규모: 319,498명 (최대)
  - 오랜 고객, 활발한 이용, 높은 지출, 낮은 이탈 이력, VIP 비중 높음  
  - **해석:** 충성 고객 다수, 프리미엄 서비스 및 이탈 방지 중요

- **군집 1: 저활동/초기 이탈 가능성 고객**  
  - 규모: 80,700명
  - 오래된 발급, 극도로 낮은 이용, 일부 탈회 이력  
  - **해석:** 비활성·유령 고객, 재활성화 캠페인 필요

- **군집 2: 신규 고객 또는 빠르게 이탈한 고객**  
  - 규모: 217,678명
  - 최근 가입, 높은 카드 신청, 높은 탈회 이력  
  - **해석:** 신규·이탈 위험, 온보딩 및 이탈 방지 강화 필요

- **군집 3: 적극적인 신규 또는 중급 이용 고객**  
  - 규모: 223,079명
  - 오랜 고객, 중간 이용, 가장 낮은 탈회 이력  
  - **해석:** 안정적 고객, VIP 성장 유도 가능

- **군집 4: 중저가 이용/평범한 고객**  
  - 규모: 159,045명 (최소)
  - 오랜 고객, 낮은 이용, 낮은 탈회 이력  
  - **해석:** 평범·잠자는 고객, 맞춤형 혜택 통한 이용 유도 필요

---

## 4. 비즈니스적 시사점 및 전략

- **군집 0:** 최우수/충성 고객 → 이탈 방지, 프리미엄 서비스 유지
- **군집 1:** 비활성/이탈 가능성 고객 → 재활성화 캠페인, 효율적 관리
- **군집 2:** 신규/초기 이탈 위험 고객 → 초기 경험 개선, 이탈 방지 프로그램
- **군집 3:** 안정적 중급 이용 고객 → VIP 성장 유도, 상향/교차 판매
- **군집 4:** 저활동 장기 고객 → 맞춤형 혜택 제공, 추가 이용 유도

---

### 💡 추가 해석
- 군집 4의 넓은 PCA 분포는 실제로 다양한 특성을 가진 '평범한' 고객이 포함되어 있기 때문일 수 있음.
- 시각적으로 넓게 퍼진 것은 차원 축소의 왜곡일 수도 있지만, 군집 4 내 데이터가 특정 특징으로 강하게 뭉치기보다 넓게 분포하는 경향을 통계적으로 보여줌.

---

**이 요약은 PCA 기반 소비 패턴, 군집별 통계, 실제 비즈니스 전략까지 한 번에 파악할 수 있도록 통합 정리한 내용입니다.**


## 5개 클러스터별 펀드 추천 유형 및 매칭 근거

| 클러스터 | 추천 펀드 유형                  | 매칭 근거                        |
|:--------:|:-------------------------------|:---------------------------------|
| **0**    | 성장형, 트렌드형, 프리미엄형    | 활동성 높음, 금액 큼, VIP 비중   |
| **1**    | 안정지향형, 실버형              | 저활동, 이탈 경험                |
| **2**    | 젊은 실속형, 실속형             | 신규, 이탈 위험, 실용성          |
| **3**    | 가족형, 혼합형, 자산배분형      | 꾸준함, 가족 가능성, 중산층      |
| **4**    | 연금형, 안정지향형              | 장기 고객, 저활동, 평범          |

---

### 클러스터별 해석 및 펀드 추천 요약

- **클러스터 0:**  
  소비·이용이 많고 VIP 비중 높음 → 프리미엄, 성장형, 트렌드형 펀드와 가장 유사 (고위험·고수익 선호)

- **클러스터 1:**  
  거의 사용 안 하거나, 이탈 경험 있음 → 안정지향, 실버, 연금형 등 보수적·안정형 상품이 적합

- **클러스터 2:**  
  신규, 이탈 위험, 실속형 → 젊은 실속형, 단기형, ETF, 중소형주 등 실용적·저위험 상품

- **클러스터 3:**  
  꾸준한 중간이용, 가족 가능성 → 가족형, 혼합형, 자산배분형 등 안정+성장 혼합 상품

- **클러스터 4:**  
  장기고객이나 활동성 낮음, 평범 → 연금형, 실버형, 안정지향형 등 장기·저위험 상품


# 분류모델


In [0]:
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql import Row

# 1. 원본 데이터에 row_id 컬럼 생성 (샘플링과 동일한 방식으로)
if 'row_id' not in df_final.columns:
    df_final = df_final.withColumn('row_id', monotonically_increasing_id())

# 2. 클러스터 결과 DataFrame 생성 (final_cluster_ids_np는 원본 데이터 순서와 반드시 일치해야 함)
rows = [Row(row_id=int(idx), cluster=int(cluster)) for idx, cluster in enumerate(final_cluster_ids_np)]
df_final_cluster = spark.createDataFrame(rows)

# 3. 원본 데이터와 클러스터 결과를 row_id 기준으로 join
df_final_with_cluster = df_final.join(df_final_cluster, on='row_id', how='left')

# 4. 결과 확인
df_final_with_cluster.select('row_id', 'cluster').show(5)


In [0]:
df_final_with_cluster.show(5)

In [0]:
from pyspark.sql import functions as F

# 1. 데이터 정렬 후 순차적 인덱스 부여 (예: 발급회원번호 기준)
df_sorted = df_final.orderBy(F.asc("발급회원번호")).withColumn("index", F.monotonically_increasing_id())

# 2. 상위 5개 회원번호 추출 (정렬 상태에서)
member_ids_head = [row['발급회원번호'] for row in df_sorted.limit(5).collect()]
cluster_ids_head = final_cluster_ids_np[:5]

# 3. 하위 5개 회원번호 추출 (정렬 상태에서)
total_count = df_sorted.count()
member_ids_tail = [row['발급회원번호'] for row in df_sorted.filter(F.col("index") >= total_count-5).collect()]
cluster_ids_tail = final_cluster_ids_np[-5:]

# 4. 결과 출력
print("[정렬된 데이터 기준]")
print("상위 5개 회원번호:", member_ids_head)
print("상위 5개 클러스터ID:", cluster_ids_head)
print("하위 5개 회원번호:", member_ids_tail)
print("하위 5개 클러스터ID:", cluster_ids_tail)


In [0]:
# final_cluster_ids_np와 df_final의 행 순서가 동일하다는 전제 하에
member_ids = [row['발급회원번호'] for row in df_final.select('발급회원번호').collect()]
rows = [Row(발급회원번호=member_id, cluster=int(cluster)) for member_id, cluster in zip(member_ids, final_cluster_ids_np)]
df_final_cluster = spark.createDataFrame(rows)


In [0]:
df_final_with_cluster = df_final.join(df_final_cluster, on='발급회원번호', how='left')
df_final_with_cluster.select('발급회원번호', 'cluster').show(5)


In [0]:
df_final_with_cluster.show(5)

In [0]:
# 클러스터별 인원수 집계
df_final_with_cluster.groupBy('cluster').count().orderBy('cluster').show()


In [0]:
import numpy as np
import torch

# 1. 원본 전체 데이터의 PCA 벡터 추출
pca_features_np = np.array([row.pca_features.toArray() for row in df_final.select("pca_features").collect()])
pca_features_tensor = torch.from_numpy(pca_features_np).float().cuda()

# 2. 기존에 학습한 클러스터 중심(centers) 사용
# kmeans_centers: (num_clusters, feature_dim)  # 이미 샘플로부터 구해진 값

# 3. 각 행별로 클러스터 할당 (유클리드 거리 기준)
assigned_clusters = torch.argmin(torch.cdist(pca_features_tensor, kmeans_centers.to(pca_features_tensor.device)), dim=1)

# 4. 결과를 numpy로 변환
final_cluster_ids_np = assigned_clusters.cpu().numpy().astype(int)

# 5. 발급회원번호와 클러스터 번호를 매칭
member_ids = [row['발급회원번호'] for row in df_final.select('발급회원번호').collect()]
rows = [Row(발급회원번호=member_id, cluster=int(cluster)) for member_id, cluster in zip(member_ids, final_cluster_ids_np)]
df_final_cluster = spark.createDataFrame(rows)

# 6. 원본 데이터와 join
df_final_with_cluster = df_final.join(df_final_cluster, on='발급회원번호', how='left')


In [0]:
df_final_with_cluster.show(5)

In [0]:
# 발급회원번호 중복 여부 확인
from pyspark.sql import functions as F

df_final_with_cluster.groupBy('발급회원번호').count().filter(F.col('count') > 1).show()


In [0]:
# 클러스터별 인원수 집계
df_final_with_cluster.groupBy('cluster').count().orderBy('cluster').show()


In [0]:
# 중복제거 중
df_final_dedup = df_final.dropDuplicates(['발급회원번호'])


In [0]:
from pyspark.sql import functions as F
df_final_dedup.groupBy('발급회원번호').count().filter(F.col('count') > 1).count()
# 결과가 0이면 중복 없음


In [0]:
#수정된 변수로 다시 예측분류 모델 시작
from pyspark.sql import Row
import numpy as np
import torch

# 1. 중복 제거된 데이터프레임 사용
pca_features_np = np.array([row.pca_features.toArray() for row in df_final_dedup.select("pca_features").collect()])
pca_features_tensor = torch.from_numpy(pca_features_np).float().cuda()

# 2. 기존에 학습한 클러스터 중심(centers) 사용
# kmeans_centers는 이미 존재한다고 가정

# 3. 각 행별로 클러스터 할당 (유클리드 거리 기준)
assigned_clusters = torch.argmin(torch.cdist(pca_features_tensor, kmeans_centers.to(pca_features_tensor.device)), dim=1)

# 4. 결과를 numpy로 변환
final_cluster_ids_np = assigned_clusters.cpu().numpy().astype(int)

# 5. 발급회원번호와 클러스터 번호를 매칭
member_ids = [row['발급회원번호'] for row in df_final_dedup.select('발급회원번호').collect()]
rows = [Row(발급회원번호=member_id, cluster=int(cluster)) for member_id, cluster in zip(member_ids, final_cluster_ids_np)]
df_final_cluster = spark.createDataFrame(rows)

# 6. 원본 데이터(df_final)와 join (중복 제거된 df_final_dedup가 아닌 원본과 join)
df_final_with_cluster = df_final.join(df_final_cluster, on='발급회원번호', how='left')

# 결과 일부 확인
df_final_with_cluster.select('발급회원번호', 'cluster').show(5)


In [0]:
# 클러스터별 인원수 집계
df_final_with_cluster.groupBy('cluster').count().orderBy('cluster').show()


In [0]:
# 컬럼명과 데이터 타입 전체 출력
for field in df_final_with_cluster.schema.fields:
    print(f"{field.name}: {field.dataType}")


In [0]:
#진짜 최종 1300만건관련 군집분석
df_final_with_cluster.show(2, truncate=False)


In [0]:
#수치형 데이터설명
from pyspark.sql import functions as F

# numeric_cols 리스트 예시
numeric_cols = [
    '입회경과개월수_신용', '최종카드발급경과월', '탈회횟수_누적', '카드신청건수', '소지카드수_유효_신용',
    '유효카드수_체크', '이용카드수_신용', '이용카드수_체크', '청구금액_기본연회비_B0M', '청구금액_제휴연회비_B0M',
    '최종탈회후경과월', '탈회횟수_발급6개월이내', '탈회횟수_발급1년이내',
    'log_입회경과개월수_신용', 'log__1순위카드이용금액', 'log__1순위카드이용건수',
    'log__2순위카드이용금액', 'log__2순위카드이용건수', '총이용금액', 'log_총이용금액',
    'VIP등급코드_idx', '연령_idx', '최상위카드등급코드_idx', 'row_id'
]

# 백틱으로 감싸는 함수
def backtick_col(col_name):
    return f"`{col_name}`"

agg_exprs = []
for col in numeric_cols:
    agg_exprs.extend([
        F.max(backtick_col(col)).alias(f"max_{col}"),
        F.min(backtick_col(col)).alias(f"min_{col}"),
        F.mean(backtick_col(col)).alias(f"mean_{col}"),
        F.expr(f'percentile_approx({backtick_col(col)}, 0.5)').alias(f"median_{col}")
    ])

# 집계 실행
cluster_stats = df_final_with_cluster.groupBy("cluster").agg(*agg_exprs)
cluster_stats.orderBy("cluster").show(truncate=False)


In [0]:
# 최빈값추가 반복
from pyspark.sql.window import Window

for col in numeric_cols:
    if col != 'cluster':
        window_spec = Window.partitionBy("cluster").orderBy(F.desc("count"))
        mode_df = (
            df_final_with_cluster.groupBy("cluster", col)
            .count()
            .withColumn("rank", F.row_number().over(window_spec))
            .filter(F.col("rank") == 1)
            .select("cluster", F.col(col).alias(f"mode_{col}"))
        )
        cluster_stats = cluster_stats.join(mode_df, on="cluster", how="left")


In [0]:
#결과확인

In [0]:
# 1. 기존 테이블이 있다면 삭제 (덮어쓰기 안전 보장)
spark.sql("DROP TABLE IF EXISTS database_pjt.df_final_with_cluster")

# 2. Delta Lake 포맷으로 관리형 테이블로 저장 (웨어하우스/SQL/ML/시각화 모두 호환)
df_final_with_cluster.write.format("delta").mode("overwrite").saveAsTable("database_pjt.df_final_with_cluster")


In [0]:
from sklearn.metrics import silhouette_score
import numpy as np

# 1. PCA 피처와 클러스터 레이블을 numpy 배열로 추출
pca_features_np = np.array([row.pca_features.toArray() for row in df_final_with_cluster.select('pca_features').collect()])
cluster_labels_np = np.array([row.cluster for row in df_final_with_cluster.select('cluster').collect()])

# 2. 샘플링 (예: 3만개)
sample_idx = np.random.choice(len(pca_features_np), 30000, replace=False)
sample_np = pca_features_np[sample_idx]
sample_labels = cluster_labels_np[sample_idx]

# 3. 실루엣 점수 계산
score = silhouette_score(sample_np, sample_labels, metric='euclidean')
print(f"Silhouette Score (Euclidean, 3만개 샘플): {score:.4f}")


In [0]:
df_final_with_cluster = spark.table("database_pjt.df_final_with_cluster")
df_final_with_cluster.show(5)


#ml 등록


In [0]:
#mlflow 설치 및 임포트
%pip install mlflow
import mlflow
import mlflow.pytorch


In [0]:
#클러스터 모델 저장(파이토치기준)
import torch

class ClusterModel(torch.nn.Module):
    def __init__(self, centers):
        super().__init__()
        self.centers = torch.nn.Parameter(centers)
        
    def forward(self, x):
        return torch.argmin(torch.cdist(x, self.centers), dim=1)

model = ClusterModel(kmeans_centers)


In [0]:
#3. MLflow로 모델 로깅 및 등록
#(실루엣 점수, 파라미터, 클러스터 중심 시각화까지 포함)


import matplotlib.pyplot as plt

with mlflow.start_run(run_name="ClusterModelRun") as run:
    # 파라미터 로깅
    mlflow.log_param("num_clusters", kmeans_centers.shape[0])
    mlflow.log_param("distance_metric", "euclidean")
    
    # 모델 등록 (레지스트리 이름 반드시 지정)
    mlflow.pytorch.log_model(
        model,
        "cluster_model",
        registered_model_name="Classification_Member_Info"
    )
    
    # (선택) 실루엣 점수 등 메트릭 기록
    mlflow.log_metric("silhouette_score", score)
    
    # (선택) 클러스터 중심 시각화
    plt.scatter(kmeans_centers[:,0].cpu(), kmeans_centers[:,1].cpu())
    plt.title("Cluster Centers")
    plt.savefig("cluster_centers.png")
    mlflow.log_artifact("cluster_centers.png")
    
    run_id = run.info.run_id

print("MLflow run_id:", run_id)

모델 버전 관리

데이터브릭스 워크스페이스 → "Models" 탭에서 버전별 관리 가능

Staging → Production 단계 승격 가능

배포 옵션

배치 추론: spark_udf 생성하여 대규모 데이터 처리 가능


In [0]:
#최종 저장 구조

#text
#dbfs:/databricks/mlflow-tracking/experiments/
  └─ <experiment_id>/
     └─ <run_id>/
        ├─ artifacts/
        │  └─ cluster_model/
        │     ├─ model.pth
        │     └─ conda.yaml
        ├─ metrics/
        └─ params/

In [0]:
#추가적인 배포옵션?
#배치추론 : spark_udf 생성해서 대규모 데이터 처리가증
cluster_udf = mlflow.pyfunc.spark_udf(spark, f"models:/{model_name}/Production")
df_with_pred = df_final.withColumn("cluster", cluster_udf("pca_features"))


모델 모니터링

입력 데이터 분포, 예측값 분포 등을 MLflow에 추가 로깅 가능

모델 드리프트 감지 설정 가능

In [0]:
#배포옵션
# 모델 로드
model_name = "Classification_Member_Info"
loaded_model = mlflow.pytorch.load_model(f"models:/{model_name}/Production")

# 배치 예측 예시
pca_tensor = torch.from_numpy(pca_features_np).float().cuda()
predictions = loaded_model(pca_tensor)

In [0]:
import torch

# 모델을 GPU로 이동
loaded_model = loaded_model.to('cuda')

# 입력 데이터도 GPU로 이동
pca_tensor = torch.from_numpy(pca_features_np).float().cuda()

# 예측 수행
predictions = loaded_model(pca_tensor)


In [0]:
# 예측 결과를 numpy 배열로 변환
pred_np = predictions.cpu().numpy()


In [0]:
# 예측결과를 df에 추가하기
from pyspark.sql import Row

member_ids = [row['발급회원번호'] for row in df_final_dedup.select('발급회원번호').collect()]
rows = [Row(발급회원번호=member_id, cluster=int(cluster)) for member_id, cluster in zip(member_ids, pred_np)]
df_pred_cluster = spark.createDataFrame(rows)

# 원본 데이터와 join
df_final_with_pred = df_final_dedup.join(df_pred_cluster, on='발급회원번호', how='left')


In [0]:
#3. 클러스터별 통계 및 분포 분석
#groupBy, agg 등을 사용해 클러스터별 인원수, 평균값, 분포 등을 분석합니다.

df_final_with_pred.groupBy('cluster').count().orderBy('cluster').show()
df_final_with_pred.groupBy('cluster').avg('총이용금액').orderBy('cluster').show()#

In [0]:
#4. 결과 저장
#분석 결과나 예측 결과를 Delta Lake 또는 Hive 테이블로 저장하여 팀원과 공유할 수 있습니다.

df_final_with_pred.write.format("delta").mode("overwrite").saveAsTable("database_pjt.df_final_with_pred")

5. 추가 작업
실루엣 점수 등 군집 품질 평가

UMAP, PCA 등 시각화

후속 ML 파이프라인 구축 및 모델 재사용, 배포 등

# 3.소비정보 데이터 클러스터링

In [0]:
!pip install kmeans-pytorch

In [0]:
from kmeans_pytorch import kmeans
from pyspark.sql import Row

In [0]:

df_use = spark.read.format("delta").table("database_pjt.3_all")
df_use.show(3)

In [0]:
%python
df_use.printSchema()
print(df_use.columns)

In [0]:
# 회원정보 전체 데이터 수 계산 -> 클러스터링 처리 연산 관련 데이터량 추정
df_use.count()

In [0]:
%python
from pyspark.sql.functions import to_date, col

# 예시 데이터
# df = spark.createDataFrame([('202401',), ('202402',), ('202403',)], ['기준년월'])

# 문자열을 datetime으로 변환 (일자를 1일로 지정)
df_use = df_use.withColumn('기준년월', to_date(col('기준년월'), 'yyyyMM'))

# datetime 타입에서 date 타입으로 변환
df_use = df_use.withColumn('기준년월', col('기준년월').cast('date'))

display(df_use)

In [0]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

# 1. 인코딩 대상 컬럼 정의
label_cols = ["_1순위업종", "_2순위업종","_3순위업종","_1순위쇼핑업종","_2순위쇼핑업종","_3순위쇼핑업종","_1순위교통업종","_2순위교통업종",
              "_3순위교통업종","_1순위여유업종","_2순위여유업종","_3순위여유업종","_1순위납부업종","_2순위납부업종","_3순위납부업종"]

# 2. Indexing + One-Hot Encoding 단계 정의
indexers = [
    StringIndexer(inputCol=col, outputCol=f"{col}_idx", handleInvalid="keep")
    for col in label_cols
]

# 3. 파이프라인 실행
pipeline = Pipeline(stages=indexers)
encoded_model = pipeline.fit(df_use)
df_encoded = encoded_model.transform(df_use)

# 4. 사용할 컬럼 정리
encoded_cols = [f"{col}_idx" for col in label_cols if f"{col}_idx" in df_encoded.columns]

original_cols = [col for col in df_use.columns if col not in (label_cols)]

# 5. 최종 df 구성
df_final = df_encoded.select(original_cols + encoded_cols)


범주형 데이터 인코딩 완료, 허나 수치형 데이터가 너무 많아서 더 쳐낼 필요가 있겠음
일단 상관관계를 다시보고 0.8 이상인 칼럼이 있는지 확인하고, 있으면 쳐냄

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
import pandas as pd
import numpy as np

# 1. 수치형 변수 리스트 추출
numeric_cols = [c for c, t in df_final.dtypes if t in ['double', 'int']]

# 2. VectorAssembler로 features 컬럼 생성
assembler = VectorAssembler(inputCols=numeric_cols, outputCol="features")
df_vec = assembler.transform(df_final).select("features")

# 3. 피어슨 상관계수 행렬 계산
corr_matrix = Correlation.corr(df_vec, "features", "pearson").head()[0].toArray()

# 4. pandas DataFrame으로 변환
corr_df = pd.DataFrame(corr_matrix, index=numeric_cols, columns=numeric_cols)

# 5. 자기 자신과의 상관관계 제외 (1.0 제거)
mask = np.triu(np.ones(corr_df.shape), k=1).astype(bool)  # 상삼각 행렬 마스크
corr_filtered = corr_df.where(mask)

# 6. 0.8 이상인 상관계수 필터링
high_corr = corr_filtered.stack().reset_index()
high_corr.columns = ['변수1', '변수2', '상관계수']
high_corr = high_corr[high_corr['상관계수'] >= 0.8]

# 7. 출력
print("상관계수가 0.8 이상인 변수 쌍:")
print(high_corr.sort_values(by='상관계수', ascending=False).round(3))

# Databricks에서는 표 형태로 보기 좋게 출력
display(high_corr.sort_values(by='상관계수', ascending=False).round(3))


In [0]:
# 상관관계 0.8 이상 헤비 컬럼 50개 삭제

delete_col = ["이용개월수_부분무이자_R12M","이용횟수_선결제_B0M","이용개월수_페이_오프라인_R6M","이용개월수_카드론_R3M","이용개월수_체크_R12M",
              "이용금액_A_페이_R6M","이용개월수_페이_온라인_R6M","이용금액_할부_무이자_B0M","이용개월수_체크_R12M","이용개월수_카드론_R12M",
              "이용개월수_카드론_R6M","이용금액_CA_R3M","이용개월수_신용_R12M","이용개월수_페이_오프라인_R6M","이용금액_신용_R3M","이용금액_B페이_R3M",
              "이용횟수_연체_R3M","이용개월수_CA_R12M","이용개월수_선결제_R6M","이용개월수_온라인_R6M","이용개월수_CA_R12M","이용금액_선결제_R3M",
              "이용횟수_연체_R6M","이용금액_CA_B0M","이용횟수_선결제_B0M","이용횟수_연체_B0M","이용개월수_D페이_R6M","이용금액_체크_R6M","이용횟수_선결제_B0M","이용개월수_전체_R6M","이용금액_할부_무이자_R6M","이용개월수_할부_유이자_R12M","이용금액_CA_R3M","이용개월수_CA_R12M","이용횟수_연체_R3M","이용개월수_선결제_R6M","이용개월수_할부_유이자_R6M","이용개월수_할부_유이자_R12M",
              "이용금액_신용_R3M","이용개월수_전체_R6M","이용개월수_신용_R12M","이용금액_C페이_B0M","이용금액_선결제_R3M","이용횟수_연체_R6M",
              "이용금액_오프라인_R6M","이용금액_B페이_B0M","이용금액_오프라인_R6M","이용금액_신용_R6M","이용금액_오프라인_R6M","이용금액_할부_무이자_R3M"]
              
df_final = df_final.drop(*delete_col)

df_final.columns

In [0]:
df_final.write.format("delta").mode("error").saveAsTable("database_pjt.3_use_encoding")

In [0]:
# 전체 데이터의 0.5% (90만개)만 추출하여 샘플링 및 저장

df_final_sample = df_final.sample(fraction=0.005, seed=42)
df_final_sample.write.format("delta").mode("error").saveAsTable("database_pjt.3_use_encoding_sample")

In [0]:
df_use_final_sample = df_final_sample.toPandas()

# PCA

In [0]:
from pyspark.ml.feature import VectorAssembler

# (1) 실제 존재하는 one-hot 컬럼만 필터링
encoded_feature_cols = [f"{col}_oh" for col in one_hot_cols]
existing_encoded_cols = [col for col in encoded_feature_cols if col in df_final.columns]

# (2) VectorAssembler 구성
assembler = VectorAssembler(
    inputCols=existing_encoded_cols,
    outputCol="features"
)

df_vectorized = assembler.transform(df_final)


In [0]:
df_final.write.format("delta").mode("error").saveAsTable("database_pjt.3_use_encoding")

In [0]:
display(df_final.limit(3))

In [0]:
import torch

# DataFrame을 numpy 배열로 변환 후 텐서로 변환
# 예: 상위 100만건만 추출
sampled_df = df_use.limit(1000000).toPandas().select_dtypes(include='number')
df_use_tensor = torch.from_numpy(sampled_df.values).float().cuda()


In [0]:
#kmeans(gpu) 샘플데이터로 클러스터링
#이거 하고 최단연결, 최장연결, 평균연결,중심연결 4개 비교도 해보고싶긴한데
from kmeans_pytorch import kmeans

# 샘플 데이터로 KMeans (유클리드)
kmeans_cluster_ids, kmeans_centers = kmeans(
    X=df_use_tensor,
    num_clusters=5,
    distance='euclidean',
    device=torch.device('cuda')
)

In [0]:
# 4. 전체 데이터 예측 (배치 처리 권장, 메모리 부족시 limit 사용)
# 4. 전체 데이터 예측 함수/ 데이터가 어느 클러스터 속하는지 예측할당
import torch
def assign_clusters(X, centers, distance='euclidean'):
    centers = centers.to(X.device)
    if distance == 'euclidean':
        dists = torch.cdist(X, centers)
    elif distance == 'cosine':
        X_norm = X / X.norm(dim=1, keepdim=True)
        centers_norm = centers / centers.norm(dim=1, keepdim=True)
        dists = 1 - torch.mm(X_norm, centers_norm.t())
    else:
        raise ValueError("지원하지 않는 distance")
    return torch.argmin(dists, dim=1)

# 5. 전체 데이터 배치 처리 (70,000개 제한 예시)
df_final_limit = df_final.limit(10000)
pca_features_np = np.array([row.pca_features.toArray() for row in df_final_limit.select("pca_features").collect()])
pca_features_tensor = torch.from_numpy(pca_features_np).float().cuda()

In [0]:
# 6. 클러스터 할당 즉 행이 어떤 클러스터 속하는지 실제 계산후 라벨링
kmeans_cluster_ids_full = assign_clusters(pca_features_tensor, kmeans_centers, 'euclidean')

In [0]:
# 7. 메타 클러스터링/ 유클리드랑 코사인거리등 보고, 통합해서 한번더 군집화
meta_features = torch.stack([kmeans_cluster_ids_full, kmeans2_cluster_ids_full], dim=1).float()
meta_cluster_ids, meta_centers = kmeans(
    X=meta_features,
    num_clusters=3,
    distance='euclidean',
    device=torch.device('cuda'),
    tol=1e-4
)

In [0]:
# 8. 결과 변환
final_cluster_ids_np = meta_cluster_ids.cpu().numpy().astype(int)
rows = [Row(id=idx, final_cluster=int(cluster)) for idx, cluster in enumerate(final_cluster_ids_np)]
df_final_cluster = spark.createDataFrame(rows)
df_final_cluster.show(5)

In [0]:
import numpy as np
from sklearn.metrics import silhouette_score

# 리미트개수에서 1,0000개만 무작위 샘플링
idx = np.random.choice(len(pca_sample_np), 30000, replace=False)
sample_np = pca_sample_np[idx]
sample_labels = kmeans_cluster_ids.cpu().numpy()[idx]

score = silhouette_score(sample_np, sample_labels, metric='euclidean')
print(f"Silhouette Score (Euclidean, 30000개 샘플): {score:.4f}")

In [0]:
# Delta Lake 관리형 테이블 불러오기
df = spark.sql("SELECT * FROM database_pjt.df_final_with_cluster")
# 또는
df = spark.table("database_pjt.df_final_with_cluster")


In [0]:
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
import numpy as np

# 예시: 군집별 중심(centroid) 및 응집도(SSE) 계산
# 1. 군집별 중심(centroid) 계산 (예: 연령, 이용금액, 탈회횟수 등 주요 변수 기준)
assembler = VectorAssembler(inputCols=["연령_idx", "이용금액", "탈회횟수_누적"], outputCol="features")
df_feat = assembler.transform(df)

# 2. 군집별 중심(centroid) 계산 (Spark SQL로 직접 계산)
centroids = df_feat.groupBy("cluster").agg(
    F.mean("연령_idx").alias("mean_age"),
    F.mean("이용금액").alias("mean_amount"),
    F.mean("탈회횟수_누적").alias("mean_churn")
).orderBy("cluster").collect()

# 3. 각 군집의 응집도(SSE, Within-Cluster Sum of Squares) 계산
# ※ Spark에서는 직접 계산이 복잡하므로, 샘플링 후 pandas로 계산하는 것이 효율적
# 아래는 pandas 예시 (Spark에서 collect 후 처리)
import pandas as pd

pdf = df_feat.sample(False, 0.01).toPandas()  # 샘플링
pdf['cluster'] = pdf['cluster'].astype(int)
centroids_pd = pd.DataFrame([(c['cluster'], c['mean_age'], c['mean_amount'], c['mean_churn']) 
                            for c in centroids], 
                           columns=['cluster', 'mean_age', 'mean_amount', 'mean_churn'])

sse = []
for cl in pdf['cluster'].unique():
    cl_data = pdf[pdf['cluster'] == cl]
    centroid = centroids_pd[centroids_pd['cluster'] == cl].iloc[0]
    dist = np.sqrt(
        (cl_data['연령_idx'] - centroid['mean_age'])**2 +
        (cl_data['이용금액'] - centroid['mean_amount'])**2 +
        (cl_data['탈회횟수_누적'] - centroid['mean_churn'])**2
    )
    sse.append(dist.sum())

print("군집별 응집도(SSE):", sse)


In [0]:
# 전체 중심(global centroid) 계산
global_centroid = pdf[['연령_idx', '이용금액', '탈회횟수_누적']].mean()

ssb = []
for cl in pdf['cluster'].unique():
    centroid = centroids_pd[centroids_pd['cluster'] == cl].iloc[0]
    n = len(pdf[pdf['cluster'] == cl])
    dist = n * (
        (centroid['mean_age'] - global_centroid['연령_idx'])**2 +
        (centroid['mean_amount'] - global_centroid['이용금액'])**2 +
        (centroid['mean_churn'] - global_centroid['탈회횟수_누적'])**2
    )
    ssb.append(dist)

print("군집별 분리도(SSB):", ssb)


In [0]:
# 예시: 군집별 소속 확률이 비슷한 데이터(중첩 영역) 비율 계산
# ※ 실제로는 군집 소속 확률이 필요하지만, hard clustering에서는 불가 → 대안: 군집 경계 근처 데이터 비율 계산

# 1. 각 데이터와 가장 가까운 두 군집 중심 거리 차이 계산
def get_closest_two(pdf, centroids_pd):
    dists = []
    for idx, row in pdf.iterrows():
        d = []
        for _, c in centroids_pd.iterrows():
            dist = np.sqrt(
                (row['연령_idx'] - c['mean_age'])**2 +
                (row['이용금액'] - c['mean_amount'])**2 +
                (row['탈회횟수_누적'] - c['mean_churn'])**2
            )
            d.append(dist)
        sorted_d = sorted(d)
        dists.append(sorted_d[1] - sorted_d[0])  # 1위와 2위 군집 중심 거리 차이
    return dists

pdf['dist_diff'] = get_closest_two(pdf, centroids_pd)

# 2. 경계 중첩 비율: 1위와 2위 군집 중심 거리 차이가 작은 데이터 비율
threshold = 0.3  # 임계값 (실제 데이터에 맞게 조정)
overlap_ratio = (pdf['dist_diff'] < threshold).mean()
print("경계 중첩 비율:", overlap_ratio)


In [0]:
df_final_with_pred = spark.table("database_pjt.df_final_with_pred")
df_final_with_pred.show(5)


In [0]:
import mlflow
import mlflow.pytorch

# 등록된 모델 이름과 버전
model_name = "Classification_Member_Info"
model_version = 1  # 필요시 변경

# MLflow에서 모델 불러오기
loaded_model = mlflow.pytorch.load_model(f"models:/{model_name}/{model_version}")


In [0]:
import torch

# 모델을 GPU로 이동 (GPU 사용 시)
loaded_model = loaded_model.to('cuda')

# 입력 데이터도 GPU로 이동
pca_tensor = torch.from_numpy(pca_features_np).float().cuda()

# 예측 수행
predictions = loaded_model(pca_tensor)
pred_np = predictions.cpu().numpy()  # numpy 배열로 변환


In [0]:
from pyspark.sql import Row

# 발급회원번호 리스트 준비 (원본 데이터프레임에서 추출)
member_ids = [row['발급회원번호'] for row in df_final_dedup.select('발급회원번호').collect()]

# 예측 결과와 발급회원번호를 묶어서 Row 리스트 생성
rows = [Row(발급회원번호=member_id, cluster=int(cluster)) for member_id, cluster in zip(member_ids, pred_np)]

# Spark DataFrame으로 변환
df_pred_cluster = spark.createDataFrame(rows)


In [0]:
from pyspark.ml.feature import VectorAssembler, PCA
import numpy as np

# 예시: 분석에 사용할 피처 리스트 (실제 컬럼명에 맞게 수정 필요)
feature_cols = [
    '입회경과개월수_신용', '탈회횟수_누적', '이용금액_R3M_신용', '이용금액_R3M_체크',
    '_1순위카드이용금액', '_1순위카드이용건수', '_2순위카드이용금액', '_2순위카드이용건수',
    '청구금액_기본연회비_B0M', '청구금액_제휴연회비_B0M', '카드신청건수', '최종카드발급경과월'
]

# 1. VectorAssembler로 피처 벡터 생성
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df_features = assembler.transform(df_final_with_pred)

# 2. PCA 모델 생성 (예: 3차원으로 축소)
pca = PCA(k=3, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(df_features)
df_pca = pca_model.transform(df_features)

# 3. PCA 결과를 numpy 배열로 변환
pca_features_np = np.array(df_pca.select("pca_features").rdd.map(lambda row: row.toArray()).collect())

# 4. pca_features_np 변수 생성 확인
print(f"pca_features_np shape: {pca_features_np.shape}")


In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline

# 예시: 투자위험등급(등급이 있다면) 또는 군집 라벨을 타겟으로 사용
# 만약 군집 라벨이 있다면, 'cluster' 컬럼을 타겟으로 사용
features = [
    '입회경과개월수_신용', '탈회횟수_누적', '이용금액_R3M_신용', '이용금액_R3M_체크',
    '_1순위카드이용금액', '_1순위카드이용건수', '_2순위카드이용금액', '_2순위카드이용건수',
    '청구금액_기본연회비_B0M', '청구금액_제휴연회비_B0M', '카드신청건수', '최종카드발급경과월'
]

assembler = VectorAssembler(inputCols=features, outputCol="features")

# 예시: 군집 라벨을 타겟으로 사용
rf = RandomForestClassifier(featuresCol="features", labelCol="cluster")

pipeline = Pipeline(stages=[assembler, rf])
model = pipeline.fit(df_final_with_pred)

# 피처 중요도 추출
rfModel = model.stages[-1]
importances = rfModel.featureImportances
for i, (col, imp) in enumerate(zip(features, importances)):
    print(f"{i+1}. {col}: {imp:.4f}")
